# Проект по SQL

## Вступление

**Цели исследования:** проанализировать базу данных крупного сервиса для чтения книг.

В нашем расположении есть база данных, содержащая информацию о книгах, авторах, издательствах, пользовательских оценках и сборах.

Разделим работу на следующие этапы:
1. Вступление
2. Загрузка данных
3. Исследовательский анализ:
        1. Посчитать, сколько книг вышло после 1 января 2000 года;
        2. Посчитать количество обзоров и среднюю оценку для каждой книги;
        3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
        4. Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками);
        5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Описание данных:

Таблица **books** содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица **authors** содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица **publishers** содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица **ratings** содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица **reviews** содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

**Схема данных:**

![avatar](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

## Загрузка данных

### Загрузим необходимые библиотеки

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

### Выведем первые 5 строк таблицы с книгами:

In [2]:
query = '''
SELECT * 
FROM books 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Выведем первые 5 строк таблицы с авторами:

In [3]:
query = '''
SELECT * 
FROM authors 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Выведем первые 5 строк таблицы с издательствами:

In [4]:
query = '''
SELECT * 
FROM publishers 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Выведем первые 5 строк таблицы с пользовательскими оценками:

In [5]:
query = '''
SELECT * 
FROM ratings 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Выведем первые 5 строк таблицы с пользовательскими обзорами:

In [6]:
query = '''
SELECT * 
FROM reviews 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследовательский анализ

### Посчитать, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT COUNT(*) number_of_books
FROM books 
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,number_of_books
0,819


**Вывод:**

После 1-го января 2000 года было выпущено 819 книг.

### Посчитать количество обзоров и среднюю оценку для каждой книги;

In [8]:
query = '''
WITH review_cnt AS (
SELECT b.book_id,
        b.title,
        COUNT(rw.review_id) review_cnt
FROM books AS b
LEFT JOIN reviews AS rw ON b.book_id = rw.book_id

GROUP BY b.book_id
ORDER BY b.book_id),

rating_avg AS (SELECT b.book_id,
        ROUND(AVG(rt.rating), 2) rating_avg
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id

GROUP BY b.book_id
ORDER BY b.book_id)

SELECT ra.book_id,
        rc.title,
        rc.review_cnt,
        ra.rating_avg
FROM rating_avg AS ra
FULL OUTER JOIN review_cnt AS rc ON ra.book_id = rc.book_id
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_cnt,rating_avg
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


**Вывод:**

Для каждой из 1000 книг выведен средний рейтинг и количество обзоров.

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;

In [9]:
query = '''
WITH publisher_cnt AS (
SELECT publisher_id,
        COUNT(*) books_cnt
FROM books
WHERE num_pages > 50
GROUP BY publisher_id)

SELECT p.publisher,
        pc.books_cnt
FROM publisher_cnt AS pc
LEFT JOIN publishers AS p ON pc.publisher_id = p.publisher_id
ORDER BY books_cnt DESC
LIMIT 1
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_cnt
0,Penguin Books,42


**Вывод:**

Наибольшее количество книг толщиной более 50 страниц выпустило издание Penguin Books, а именно 42 книги.

### Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками);

In [10]:
query = '''
WITH rating_query AS (
SELECT book_id,
        COUNT(*) rating_cnt
FROM ratings
GROUP BY book_id
HAVING COUNT(*) >= 50
ORDER BY rating_cnt DESC)

SELECT a.author,
        ROUND(AVG(r.rating), 2) avg_rating
FROM books AS b
RIGHT JOIN rating_query AS rq ON b.book_id = rq.book_id
LEFT JOIN authors AS a ON a.author_id = b.author_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:**

J.K. Rowling/Mary GrandPré имеет самую высокую оценку среди авторов книг с 50 и более оценками.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = '''
WITH user_ratings AS (
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48
),

user_reviews AS (
SELECT username, 
        COUNT(review_id) AS review_cnt
FROM reviews
WHERE username IN (SELECT username 
                    FROM user_ratings)
GROUP BY username
)

SELECT AVG(review_cnt) AS avg_reviews
FROM user_reviews
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_reviews
0,24.0


**Вывод:**

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора